In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

## Исходный вариант:

In [428]:
votes = pd.read_csv("train_data.csv")

In [429]:
answers = pd.read_csv("train_answers.csv")

In [255]:
quorum = votes.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()

In [ ]:
data = quorum.merge(answers, on=["itemId"])

In [ ]:
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)

In [ ]:
data["iou"].mean()

## Рассмотрим другие оценки: усеченное среднее и медиану, как менее чувствительные к выбросам

In [431]:
from scipy.stats import trim_mean

quorum = votes.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].agg(lambda x: trim_mean(x, 0.1)).reset_index()
data = quorum.merge(answers, on=["itemId"])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.5064341810007029

In [432]:
quorum = votes.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].median().reset_index()
data = quorum.merge(answers, on=["itemId"])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.5154863220078157

Качество немного лучше исходного варианта, но по-прежнему невысокое

## Рассмотрим вариант с наибольшим захватом отмеченных пользователями границ

In [433]:
votes = pd.read_csv('train_data.csv')
quorum = votes.groupby('itemId')[['Xmin','Ymin', 'Xmax', 'Ymax']].agg({'Xmin': 'min', 'Xmax': 'max', 'Ymin': 'min', 'Ymax': 'max'}).reset_index()
data = quorum.merge(answers, on=["itemId"])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.5809800227327839

Результат лучше, однако на тестовой выборке качество низкое

## Рассмотрим версии, если данные пользователей в основном охватывают большую (меньшую) область, чем нужно

In [437]:
votes = pd.read_csv('train_data.csv')
quorum = votes.groupby('itemId')[['Xmin','Ymin', 'Xmax', 'Ymax']].agg({'Xmin': lambda x: np.quantile(x, 0.4),
                                                                      'Xmax': lambda x: np.quantile(x, 0.6),
                                                                      'Ymin': lambda x: np.quantile(x, 0.4),
                                                                      'Ymax': lambda x: np.quantile(x, 0.6)}).reset_index()
data = quorum.merge(answers, on=["itemId"])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.5343152796052226

In [436]:
votes = pd.read_csv('train_data.csv')
quorum = votes.groupby('itemId')[['Xmin','Ymin', 'Xmax', 'Ymax']].agg({'Xmin': lambda x: np.quantile(x, 0.6),
                                                                      'Xmax': lambda x: np.quantile(x, 0.4),
                                                                      'Ymin': lambda x: np.quantile(x, 0.6),
                                                                      'Ymax': lambda x: np.quantile(x, 0.4)}).reset_index()
data = quorum.merge(answers, on=["itemId"])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
data["iou"].mean()

0.4921461017812395

Хотя результаты на тренировачной выборки лучше, на тестовой качество снова низкое

Также была рассмотрена линейная регрессия для предсказания значения координаты по остальным признакам, она не приведена в отчете из-за низкого качества

Были еще попытки оценивать положение по среднему соотношению размеров сторон и среднему положению центра прямоугольников, выделенных пользователями, в этом случае опять же качество было низким.

Из всех рассмотренных вариантов лучше всего использовать медиану: выше качество модели и ниже чувствительность к выбросам.